In [ ]:
import logging

import config
import polars as pl
from lifelines import KaplanMeierFitter

from src.customs.fold import add_kfold
from src.customs.metrics import Metric
from src.model.sklearn_like import WeightedAverageModel, WeightedAverageModelWrapper
from src.trainer.tabular.simple import single_inference_fn, single_train_fn

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

In [ ]:
class CreateTargetFn:
    def __init__(self, time_col: str, event_col: str):
        self.time_col = time_col
        self.event_col = event_col
        self.target_cols = [
            "target_kmf_race_group",
            "target_kmf_race_group_with_penalty",
            "target_kmf",
            "target_kmf_with_penalty",
            "target_xgboost_cox",
        ]

    def __call__(self, df: pl.DataFrame) -> pl.DataFrame:
        # add kmf target by race_group
        dfs = []
        for _, i_df in df.group_by("race_group", maintain_order=True):
            kmf = KaplanMeierFitter()
            kmf.fit(i_df[self.time_col], event_observed=i_df[self.event_col])
            _df = i_df.with_columns(
                pl.Series("target_kmf_race_group", kmf.survival_function_at_times(i_df[self.time_col])),
            )
            dfs.append(_df)
        target_df = pl.concat(dfs)

        # with penalty (https://www.kaggle.com/code/abdmental01/abdbase-single-cat-cv-0-679-lb-0-686)
        target_df = target_df.with_columns(
            pl.when(target_df[self.event_col] == 0)
            .then(target_df["target_kmf_race_group"] - 0.15)
            .otherwise(target_df["target_kmf_race_group"])
            .alias("target_kmf_race_group_with_penalty"),
        )

        # add kmf target
        kmf = KaplanMeierFitter()
        kmf.fit(target_df[self.time_col], event_observed=target_df[self.event_col])
        target_df = target_df.with_columns(
            pl.Series("target_kmf", kmf.survival_function_at_times(target_df[self.time_col])),
        )

        target_df = target_df.with_columns(
            pl.when(target_df[self.event_col] == 0)
            .then(target_df["target_kmf"] - 0.15)
            .otherwise(target_df["target_kmf"])
            .alias("target_kmf_with_penalty"),
        )

        # add xgboost cox target
        target_df = target_df.with_columns(
            pl.when(target_df[self.event_col] == 0)
            .then(-1 * target_df[self.time_col])
            .otherwise(target_df[self.time_col])
            .alias("target_xgboost_cox"),
        )

        return target_df


def load_data():
    raw_train_df = pl.read_csv(config.COMP_DATASET_DIR / "train.csv").with_columns(
        pl.lit("TRAIN").alias(config.DATASET_COL),
        pl.lit(-1).alias(config.FOLD_COL),
    )

    create_target_fn = CreateTargetFn(time_col=config.SURVIVAL_TIME_COL, event_col=config.EVENT_COL)
    raw_train_df = create_target_fn(raw_train_df)

    raw_test_df = pl.read_csv(config.COMP_DATASET_DIR / "test.csv").with_columns(
        pl.lit("TEST").alias(config.DATASET_COL)
    )

    config.META_COLS = sorted(list(set(config.META_COLS + create_target_fn.target_cols)))

    train_test_df = pl.concat([raw_train_df, raw_test_df], how="diagonal_relaxed").sort(
        config.DATASET_COL, config.ID_COL, descending=[True, False]
    )
    return train_test_df


def load_ensemble_source(
    train_test_df: pl.DataFrame,
    exp_names: dict[str, str],
    source_prefix: str = "",
    output_dataset: str = "TRAIN",
) -> pl.DataFrame:
    te_result_path = (  # noqa
        lambda exp_name, model_name: config.ARTIFACT_EXP_DIR(exp_name) / model_name / "te_result.csv"
        if not config.IS_KAGGLE_ENV
        else config.OUTPUT_DIR / exp_name / model_name / "te_result.csv"
    )

    source_dfs = [
        pl.concat(
            [
                pl.read_csv(
                    config.ARTIFACT_EXP_DIR(exp_name) / model_name / "va_result.csv",
                    columns=[config.ID_COL, "pred"],
                )
                .with_columns(pl.col("pred").rank().alias(f"pred_rank_{exp_name}"))
                .rename({"pred": f"pred_{exp_name}"}),
                pl.read_csv(te_result_path(exp_name, model_name), columns=[config.ID_COL, "pred"])
                .with_columns(pl.col("pred").rank().alias(f"pred_rank_{exp_name}"))
                .rename({"pred": f"pred_{exp_name}"}),
            ],
            how="diagonal_relaxed",
        )
        for exp_name, model_name in exp_names.items()
    ]
    source_df = source_dfs.pop(0)
    for df in source_dfs:
        source_df = source_df.join(df, on=config.ID_COL, how="left")

    # add prefix
    source_df = source_df.select(
        [
            config.ID_COL,
            pl.all().exclude(config.ID_COL).name.prefix(source_prefix),
        ]
    )
    return train_test_df.join(source_df, on=config.ID_COL, how="left").filter(
        pl.col(config.DATASET_COL) == output_dataset
    )

In [ ]:
train_test_df = load_data()
features_df = load_ensemble_source(train_test_df, config.ENSEMBLE_EXP_NAMES, source_prefix=config.FEATURE_PREFIX)

feature_names = sorted([x for x in features_df.columns if x.startswith(config.FEATURE_PREFIX)])
feature_names = [x for x in feature_names if x.startswith(f"{config.FEATURE_PREFIX}pred_rank_")]

cat_features = [x for x in feature_names if x.startswith(f"{config.FEATURE_PREFIX}c_")]

logger.info(f"# of features: {len(feature_names)}")
logger.info(f"# of cat_features: {len(cat_features)}")

In [ ]:
train_features_df = add_kfold(
    features_df,
    n_splits=config.N_SPLITS,
    random_state=config.N_SPLITS,
    fold_col=config.FOLD_COL,
)

va_result_df, va_scores, trained_models = single_train_fn(
    model=WeightedAverageModelWrapper(
        name="ave",
        model=WeightedAverageModel(weights=None),
        feature_names=feature_names,
    ),
    features_df=train_features_df,
    feature_cols=feature_names,
    target_col="target_kmf_with_penalty",
    fold_col=config.FOLD_COL,
    meta_cols=config.META_COLS,
    out_dir=config.OUTPUT_DIR,
    train_folds=None,
    eval_fn=Metric(),
    overwrite=True,
    use_eval_metric_extra_va_df=False,
    enable_plot_feature_importance=False,
)

In [ ]:
# debug test
test_features_df = load_ensemble_source(
    train_test_df,
    config.ENSEMBLE_EXP_NAMES,
    source_prefix=config.FEATURE_PREFIX,
    output_dataset="TEST",
)

te_result_df = single_inference_fn(
    model=WeightedAverageModelWrapper(
        name="ave",
        model=WeightedAverageModel(weights=None),
        feature_names=feature_names,
    ),
    features_df=test_features_df,
    feature_names=feature_names,
    model_dir=config.ARTIFACT_EXP_DIR(),
    out_dir=config.OUTPUT_DIR,
    inference_folds=list(range(config.N_SPLITS)),
)
print(te_result_df["pred"].to_list())